In [ ]:
!tail /content/layers.py

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Dropout, Bidirectional, LSTM, BatchNormalization, GRU

from tensorflow.keras import regularizers
from ds_utils.layers import SeqWiseBatchNorm, RowConv1D

import tensorflow_datasets as tfds

import numpy as np
from matplotlib import pyplot as plt

print(tf.__version__)

2.2.0


In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
# Loading the dataset
(ds_train, ds_test), ds_info = tfds.load(
    'imdb_reviews',
    split=['train', 'test'],
    shuffle_files=True,
    as_supervised=True,
    with_info=True,
)

In [ ]:
# Prepping the data a little
x_train, y_train, x_test, y_test = [], [], [], []

for x in ds_train:
  text = str(x[0].numpy()).split("b", 1)[1][1:-1]
  label = x[1].numpy()
  x_train.append(text)
  y_train.append(label)

for x in ds_test:
  text = str(x[0].numpy()).split("b", 1)[1][1:-1]
  label = x[1].numpy()
  x_test.append(text)
  y_test.append(label)

x_train, y_train, x_test, y_test = np.array(x_train), np.array(y_train), np.array(x_test), np.array(y_test)

print(x_train.shape, y_train.shape)

In [ ]:
vocab_size = 80000

tokenizer = Tokenizer(num_words = vocab_size, lower = True, oov_token = '<OOV>')
tokenizer.fit_on_texts(x_train)


In [ ]:
maxlen = 100
padding = 'post'
truncating = 'post'

sequences = tokenizer.texts_to_sequences(x_train)
seq_train = pad_sequences(sequences, maxlen = maxlen, padding = padding, truncating = truncating)

test_sequences = tokenizer.texts_to_sequences(x_test)
seq_test = pad_sequences(test_sequences, maxlen = maxlen, padding = padding, truncating = truncating)

In [2]:
embedding_dim = 300

model = Sequential([
                    Embedding(input_dim = 80000+1, output_dim= embedding_dim, input_length = 100),
                    LSTM(64, kernel_regularizer=regularizers.l2(0.001), return_sequences=True),
                    (LSTM(64, kernel_regularizer=regularizers.l2(0.001), return_sequences=True)),
                     
                    # RowConv1D(window_size = 2, strides = 2), 
                    # Dense(64, activation = 'relu', kernel_regularizer='l2'),
                    # Dropout(0.6),
                    # Dense(32, activation = 'relu', kernel_regularizer='l2'),
                    # Dropout(0.6),
                    # Dense(1, activation = 'sigmoid')
])

model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model.summary()

init 1


TypeError: in user code:

    /home/abhay/Projects/ConvoBot_Group/Speech Recognition/ds_utils/layers.py:56 call  *
        out = tf.nn.depthwise_conv2d(I, W, strides)

    TypeError: depthwise_conv2d_v2() missing 1 required positional argument: 'padding'


In [ ]:
history = model.fit(seq_train, y_train, epochs = 1, batch_size = 100)

In [ ]:
# acc=history.history['accuracy']
# val_acc=history.history['val_accuracy']
# loss=history.history['loss']
# val_loss=history.history['val_loss']

# epochs=range(len(acc)) 

# plt.plot(epochs, acc, 'r')
# plt.plot(epochs, val_acc, 'b')
# plt.title('Training and validation accuracy')
# plt.xlabel("Epochs")  
# plt.ylabel("Accuracy")
# plt.legend(["Accuracy", "Validation Accuracy"])
# plt.show()

# plt.plot(epochs, loss, 'r')
# plt.plot(epochs, val_loss, 'b')
# plt.title('Training and validation Loss')
# plt.xlabel("Epochs")
# plt.ylabel("Loss")
# plt.legend(["Loss", "Validation Loss"])
# plt.show()

In [ ]:
# model2 = Sequential(name = "test_model", 
#                     layers = [
#                               Embedding(input_dim = vocab_size+1, output_dim= embedding_dim, input_length = maxlen),
#                               Bidirectional(GRU(64, kernel_regularizer=regularizers.l2(0.001), unroll = True, return_sequences=True))
#                     ])
# model2.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
# model2.summary()

In [ ]:
i=model.predict(seq_test)

In [ ]:
i.shape

In [ ]:
# # Extracting the Embeddings
# embedding_layer = model.layers[0]
# weights = embedding_layer.get_weights()[0]
# print(weights.shape)

In [ ]:
# word_index = {e:i for e,i in tokenizer.word_index.items() if i <= vocab_size}

In [ ]:
# # Saving the weights
# import io

# out_v = io.open('vecs.tsv', 'w', encoding='utf-8')
# out_m = io.open('meta.tsv', 'w', encoding='utf-8')

# for word in word_index:
#   vec = weights[ int(word_index[word]) - 1] # skip 0, it's padding.
#   out_m.write(word + "\n")
#   out_v.write('\t'.join([str(x) for x in vec]) + "\n")
# out_v.close()
# out_m.close()

In [ ]:
# # Downloading the weights

# try:
#   from google.colab import files
# except ImportError:
#    pass
# else:
#   files.download('vecs.tsv')
#   files.download('meta.tsv')